# Process Nov 8, 2022 election data.

In [2]:
import json

import pandas as pd

with open("2022.json", "rb") as fp:
    results = json.load(fp)

Check the office that each one of the results is for.

In [3]:
for idx, result in enumerate(results):
    print(f"{idx}: {result['office']}")

0: Governor
1: U.S. House
2: U.S. House
3: U.S. House
4: U.S. House
5: U.S. House
6: U.S. House
7: U.S. House
8: U.S. House
9: U.S. House
10: U.S. House
11: U.S. House
12: U.S. House
13: U.S. House
14: Attorney General
15: State Senate
16: State Senate
17: State Senate
18: State Senate
19: State Senate
20: State Senate
21: State Senate
22: State Senate
23: State Senate
24: State Senate
25: State Senate
26: State Senate
27: State Senate
28: State Senate
29: State Senate
30: State Senate
31: State Senate
32: State Senate
33: State Senate
34: State Senate
35: State Senate
36: State Senate
37: State Senate
38: State Senate
39: State Senate
40: State Senate
41: State Senate
42: State Senate
43: State Senate
44: State Senate
45: State Senate
46: State Senate
47: State Senate
48: State Senate
49: State Senate
50: State Senate
51: State Senate
52: State Senate
53: State House
54: State House
55: State House
56: State House
57: State House
58: State House
59: State House
60: State House
61: Sta

Alias

In [4]:
gov22 = results[0]
ag22 = results[14]
prop1 = results[163]
prop2 = results[164]
prop3 = results[165]
sos22 = results[166]

In [5]:
pd.DataFrame

pandas.core.frame.DataFrame

In [6]:
def race_to_df(race, col_prefix):
    data = list()
    for county in race["reporting_units"]:
        if county["level"] == "state":
            continue

        row = dict()
        row["county"] = county["name"]
        row[f"{col_prefix}_total"] = county["total_votes"]
        for options in county["candidates"]:
            key = f'{col_prefix}_{options["nyt_id"].split("-")[0]}'
            row[key] = options["votes"]["total"]
            key = f'{col_prefix}_pct_{options["nyt_id"].split("-")[0]}'
            row[key] = options["votes"]["total"] / county["total_votes"]
        data.append(row)
    return pd.DataFrame(data)

In [7]:
df_gov22 = race_to_df(gov22, "gov_22")
df_sos22 = race_to_df(sos22, "sos_22")
df_ag22 = race_to_df(ag22, "ag_22")

In [8]:
def prop_to_df(race):
    data = list()
    prefix = f'prop{race["seat"].split(" ")[0]}_22'
    for county in race["reporting_units"]:
        if county["level"] == "state":
            continue
        # Props should only be a yes/no vote.
        assert len(county["candidates"]) == 2
        row = dict()
        # County Name
        row["county"] = county["name"]
        # Total Votes
        row[f"{prefix}_total"] = county["total_votes"]

        for candidate in county["candidates"]:
            vote = candidate["nyt_id"]
            row[f"{prefix}_{vote}"] = candidate["votes"]["total"]
            row[f"{prefix}_{vote}_pct"] = (
                row[f"{prefix}_{vote}"] / row[f"{prefix}_total"]
            )

        data.append(row)
    return pd.DataFrame(data)

In [9]:
df_prop1 = prop_to_df(prop1)
df_prop2 = prop_to_df(prop2)
df_prop3 = prop_to_df(prop3)

In [10]:
df = df_gov22.merge(df_sos22, on="county")
df2 = df.merge(df_ag22, on="county")
df3 = df2.merge(df_prop1, on="county")
df4 = df3.merge(df_prop2, on="county")
df5 = df4.merge(df_prop3, on="county")

In [11]:
df5.to_feather("2022.ft")